# Tips for installing isaac lab
## using zsh to install the isaaclab
- Inside the `/isaacsim/setup_conda_env.sh` Removing the export line in setup_conda_env.sh
    ```shell
        #!/bin/bash
        SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
        if [ -n "$ZSH_VERSION" ]; then
            SCRIPT_DIR="$( cd "$( dirname "$0" )" && pwd )"
        -   export BASH_SOURCE=$SCRIPT_DIR/setup_python_env.sh
        fi
        MY_DIR="$(realpath -s "$SCRIPT_DIR")"
    ```
- Inside the `/isaacsim/setup_python_env.sh`
    ```shell
        - SCRIPT_DIR="$(dirname "${BASH_SOURCE}")"
        + SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
    ```
- Inside the isaaclab.sh
    ```shell
                printf '%s\n' \
                    '# for Isaac Sim' \
        -           'source '${isaacsim_setup_conda_env_script}'' \
        +           'cd '${ISAACLAB_PATH}'/_isaac_sim' \
        +           'source setup_conda_env.sh' \
        +           'cd $OLDPWD' \
                    '' >> ${CONDA_PREFIX}/etc/conda/activate.d/setenv.sh
            fi
    ```
- **Optional change**
- replacing line 19 of isaaclab.sh
    ```Shell
       export ISAACLAB_PATH="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
    ``` 
    to
    
    ```Shell
        if [ -n "$ZSH_VERSION" ]; then
            export ISAACLAB_PATH="$( cd "$( dirname "$0" )" &> /dev/null && pwd )"
        else
            export ISAACLAB_PATH="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
        fi
    ```

- And replacing the line
```
    SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
```
in `_isaac_sim/setup_conda_env.sh` and `_isaac_sim/setup_python_env.sh` to
```
    # Determine the script directory
    if [ -n "$ZSH_VERSION" ]; then
        SCRIPT_DIR="$( cd "$( dirname "$0" )" && pwd )"
    else
        SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
    fi
```
- **After modification, you need to ==restart the terminal== to install the isaaclab again**

## using isaaclab to run the isaac sim
- Inside the isaaclab path, run `source _isaac_sim/setup_conda_env.sh` to connect the isaac sim env into conda env

# Using pdb attach to debug the command from terminal
- Create a python debugger .launch.json file to config like below
  ```JSON
    {
        // Use IntelliSense to learn about possible attributes.
        // Hover to view descriptions of existing attributes.
        // For more information, visit: https://go.microsoft.com/fwlink/?linkid=830387
        "version": "0.2.0",
        "configurations": [
            {
                "name": "Python Debugger: Remote Attach",
                "type": "debugpy",
                "request": "attach",
                "connect": {
                    "host": "localhost" # the host ip,
                    "port": 5678 # the port you are listening
                },
                "pathMappings": [
                    {
                        "localRoot": "${workspaceFolder}",
                        "remoteRoot": "."
                    }
                ]
            }
        ]
    }
  ```
- Install the debugpy into your virtual env by `pip install debugpy`
- Then you can run the python file from terminal and start the debug from vscode
- `python -m debubgpy --listen <host_ip>:<port> --wait-for-client <file_names> [args]`

# Project structure for rl framework in isaac lab

## repository file structure for isaac lab
- RL algorithm: ./rsl_rl/
- Project in simulation for training: ./project_name

- Robot asset folder: ./project_name/assets Under each robot tasks foloder, you will have asset to contains the robot description and the corresponding actuator, sensor cfg for the specific task, Inside the asset python file, we will implement the actuator config based on the omni.isaac.lab.actuators.DCMotorCfg and omni.isaac.lab.assets.articulation.ArticulationCfg

- taks folder: ./project_name/locomotion (Under each taks is the env for rl training). Inside the task folder, we will implement all kinds of the training related stuff such as the observation, actions, rewards, terminatons, domain randomization, and curriculum. Most of the training related cfg is based on the mdp

- Terrian: ./project_name/terrian: we will implement terrian genertion config python file based on omni.isaac.lab.terrains, omni.isaac.lab.terrains.terrain_generator_cfg.TerrainGeneratorCfg



## Some engineering tracks:
- [urdf file visualization](https://gkjohnson.github.io/urdf-loaders/javascript/example/bundle/)
- urdf2usd: use isaacsim to import the urdf and change several parameters including the material and the like, then we could directly save selected to export the usd file. Usd file contains the stl file in one file, so the file size will be around MB instead of kb
- urdf2mjcf by mujoco
- mjcf2urdf by pybullet
- Please add flag video to save the short clip of the training video to see whether the reward is correctly configured.

# rsl-rl isaac lab debug conclusion
## manager based stuff
    - just modify the function based on the corresponding manager cfg inside the func / mdp
    - env.scene.sensors to get the sensor data from the sim
## Custom env creation
    - ManagerBasedEnv is the class for isaac sim wrapped gym api
    - modify the init(), step(), and reset() func for training 
    - reset func logic:
        1. compute curriculum based on CurriCulmManager
        2. curriculm term function is based on the mdp

    - observation manager and action manager, reward manager termination_manager

Motion retargeting technical backgrounds
- CMU H2O->OmniH2O->Hover WBC
